# GOOGLE DRIVE GRANT

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
ls "/content/drive/MyDrive/DATASET"

 DATASET_ADAMALIK.csv  'DATA TEST'/   FITUR/  'Pengukuran Darah'/


#PPG MODELING USING ANDROID

## PPG MODELING 


In [5]:
import cv2
import numpy as np
from scipy import signal
import os
import shutil

overall_results = []

def HeartRateFinger(Video_Path):
    os.makedirs("frames")  #make a directory called frames to save the images in
    vidcap = cv2.VideoCapture(Video_Path)  #the directory path to the video of interest
    images = []

    #take in images and get indidvual frames
    def getFrame(sec):
        vidcap.set(cv2.CAP_PROP_POS_MSEC, sec * 1000)  #VideoCaptureProperties to capture 
        hasFrames, image = vidcap.read()
        if hasFrames:
            cv2.imwrite("frames/image" + str(count) + ".jpg", image)  # save frame as JPG file
        images.append("frames/image" + str(count) + ".jpg")
        return hasFrames

    #set up frame rate
    sec = 0
    frameRate = 0.05  #capture frame each 0.05 seconds --> (1 second) / .05 = 20 seconds 20 fps
    count = 1
    success = getFrame(sec)  #success tracker
    while (success):
        count = count + 1
        sec = sec + frameRate
        sec = round(sec, 2)
        success = getFrame(sec)

    #Get the channels from images and Find the average of RED channel for each frame
    pixels_averages = []
    for i in range(len(images) - 1):
        bgr_image = cv2.imread(images[i])
        blue_channel, green_channel, red_channel = cv2.split(bgr_image)  #splits into the 3 color channels (RGB)
        average = np.mean(red_channel)  #mean of all the red channels  is the sum all pixels in channel and divide by number of pixels
        pixels_averages.append(average)  #add this average to all the other frames array
    pixels_averages = np.divide(np.array(pixels_averages), 255)  #normalize our averages between 0-1

    #Clean and filter data (butterworth filter)
    sos = signal.butter(4, 1, 'hp', fs=20, output='sos')  # setting up signal.butter(kind of frequency,framerate, name of method)
    filtered = signal.sosfilt(sos, pixels_averages) #Applying filter

    # truncate wonky data from  beginning of series
    filtered = filtered[40:]

    #Perform FFT to find frequency of max amplitude
    Sample_rate = 20  #same as the frame rate is the sampling rate
    BW = Sample_rate / 2  #bandwidth (range of frequency) in signal processing
    fft = np.absolute(np.fft.fft(filtered))
    frames_len = len(filtered)
    frequancies = np.arange(0, BW, Sample_rate / frames_len)
    fft = fft[0:len(frequancies)]

    #Convert back to BPM from hz
    heartrate = np.round(frequancies[np.argmax(fft)] * 60)  #take the highest peak frequency and multiply by 60 and round it
    shutil.rmtree("frames")  #remove folder made to keep the directory clean
    overall_results.append(heartrate)
    return heartrate



In [6]:
#script expected outcome while Resting is 74-78 BPM
path = "/content/drive/MyDrive/DATASET/Pengukuran Darah/95.MOV"
my_heart_rate = HeartRateFinger(path)
print("Your Heart rate is : {} bpm".format(my_heart_rate))

Your Heart rate is : 77.0 bpm


In [7]:
#script expected outcome while Resting is 69-77 BPM
path = "/content/drive/MyDrive/DATASET/resting1.mp4"
my_heart_rate = HeartRateFinger(path)
print("Your Heart rate is : {} bpm".format(my_heart_rate))

ValueError: ignored

In [ ]:
#script expected outcome while Resting is 69-77 BPM
path = "/content/drive/MyDrive/DATASET/RESTING2.mp4"
my_heart_rate = HeartRateFinger(path)
print("Your Heart rate is : {} bpm".format(my_heart_rate))

In [ ]:
#script expected outcome after activity was 83-88 BPM (50 jumping jacks)
path = "/content/drive/MyDrive/DATASET/active1.mp4"
my_heart_rate = HeartRateFinger(path)
print("Your Heart rate is : {} bpm".format(my_heart_rate))

In [ ]:
#script expected outcome after activity was 75-85 BPM (jogging for a bit)
path =  "/content/drive/MyDrive/DATASET/active2.mp4"
my_heart_rate = HeartRateFinger(path)
print("Your Heart rate is : {} bpm".format(my_heart_rate))

In [ ]:
#Overall results laid out with their error rate as well
print(overall_results)


In [ ]:
 # Command untuk menghapus file frames jika terjadi ERROR :
 "Jika terjadi Error pada saat melakukan Running Program"
 !rm -rf frames

In [ ]:
plt.plot(cD)

# DATASET PASIEN *RSUP HAJI ADAM MALIK MEDAN


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pywt

In [ ]:
datas = pd.read_excel("/content/drive/MyDrive/DATASET/Pengukuran Darah/DATASET_ADAMALIK.xlsx")

In [ ]:
datas.head()

,No,Nama Sample,Jenis Kelamin,Umur,Tekanan Darah (TD),Respiration Rate (RR),Heart Rate (HR),Saturasi Oksigen (SpO2)
0,1,M.Hut,Perempuan,62,122/66,36,132,0.83
1,2,D.Gin,Laki- Laki,43,103/10.8,14,145,0.98
2,3,SAH,Perempuan,60,105/71,20,110,0.98
3,4,M.SYA,Laki- Laki,31,97/74,32,97,0.99
4,5,A SIB,Laki- Laki,55,103/80,28,71,0.97


In [ ]:
#df1 = datas[datas.columns[6:8]] #Irisan pertama
df1 = datas[datas.columns[6:8]]

In [ ]:
df1.head()

,Heart Rate (HR),Saturasi Oksigen (SpO2)
0,132,0.83
1,145,0.98
2,110,0.98
3,97,0.99
4,71,0.97


#FEATURE EXTRACTION

## Discrete Wavelet Transform features

In [ ]:
df1.plot()
# df1 = np.array(df1)
#plt.figure(figsize=(20,5))
arr = df1.to_numpy()
# plt.plot(df1)
# plt.show()

In [ ]:
# print(arr)

In [ ]:
cA, cD = pywt.dwt(arr, 'db2')

In [ ]:
# df2.plot()
# df2 = pd.DataFrame(arr2)
# print(arr)
# print(cA)

In [ ]:
# print(cD)

In [ ]:
# w = pywt.Wavelet('sym3')

In [ ]:
# cA, cD = pywt.dwt(df1, wavelet=w, mode='constant')

In [ ]:
# int(pywt.dwt_coeff_len(data_len=len(df1), filter_len=w.dec_len, mode='symmetric'))

In [ ]:
plt.plot(cA)

#FITUR ENTROPY


In [1]:
df1.plot()
# df1 = np.array(df1)
#plt.figure(figsize=(20,5))
arr = df1.to_numpy()
# plt.plot(df1)
# plt.show()

NameError: ignored

In [2]:
# print(arr)